In [1]:
import numpy as np
import pandas as pd

import random
import os
import joblib

from sklearn.base import BaseEstimator
from sklearn.dummy import DummyRegressor

from sklearn.ensemble import RandomForestRegressor

from sklearn.utils.validation import check_X_y, check_array, check_is_fitted, column_or_1d
from sklearn.model_selection import cross_val_score, GridSearchCV, cross_val_predict

from tscv import GapKFold

from time import time, localtime, strftime



%matplotlib inline

The aim is to build a training pipeline and apply it on a small number of (building, meter)

We won't use a scikitlearn pipeline as it does not allow to clean data by deleting rows.<br>
(as explained here : https://stackoverflow.com/questions/25539311/custom-transformer-for-sklearn-pipeline-that-alters-both-x-and-y)

In [2]:
test_df = pd.read_csv('../data/raw/csvs/test.csv', parse_dates=['timestamp'])

In [3]:
test_df.head()

,row_id,building_id,meter,timestamp
0,0,0,0,2017-01-01
1,1,1,0,2017-01-01
2,2,2,0,2017-01-01
3,3,3,0,2017-01-01
4,4,4,0,2017-01-01


In [4]:
bdata = pd.read_csv(
    '../data/raw/csvs/building_metadata.csv', 
    index_col='building_id', 
    usecols=['building_id', 'site_id']
)
bdata.head()

,site_id
building_id,
0,0
1,0
2,0
3,0
4,0


In [5]:
test_df = test_df.join(bdata, on='building_id', how='left')

In [6]:
test_df.head()

,row_id,building_id,meter,timestamp,site_id
0,0,0,0,2017-01-01,0
1,1,1,0,2017-01-01,0
2,2,2,0,2017-01-01,0
3,3,3,0,2017-01-01,0
4,4,4,0,2017-01-01,0


<b>Pipeline functions</b>

In [7]:
def load_and_prepare_site_data(site_id, data_folder_path):
    
    # Loads weather data
    raw_df_weather = pd.read_csv(data_folder_path + 'weather_train.csv', 
                     parse_dates=['timestamp'], index_col=['site_id', 'timestamp'])

    b_df_weather = raw_df_weather.loc[(site_id,)]

    # keep only air_temperature and dew_temperature
    b_df_weather.drop(
        ['precip_depth_1_hr', 'sea_level_pressure', 'wind_direction', 'wind_speed', 'cloud_coverage'],
        axis=1,
        inplace=True
    )

    # Clean timestamps index.
    clean_index = pd.date_range(start=b_df_weather.index.min(), end=b_df_weather.index.max(), freq='H')
    b_df_weather = b_df_weather.reindex(index=clean_index, copy=True)


    # Interpolate missing values.
    b_df_weather.interpolate(method='linear', limit=3, inplace=True)

    # Build time features
    b_df_weather['day_hour'] = b_df_weather.index.to_series().dt.hour
    b_df_weather['day_of_week'] = b_df_weather.index.to_series().dt.dayofweek

    # Builds averaged weather features.

    timeframes = [24]
    features_to_avg = ['air_temperature', 'dew_temperature']
    do_center = False

    for c in features_to_avg:
        ts = b_df_weather[c]
        for timeframe in timeframes:
            shifted_ts = ts.rolling(timeframe, center=do_center).mean()
            new_col_name = '' + c + '_ma_' + str(timeframe) + 'H'
            b_df_weather[new_col_name] = shifted_ts
            
            
    # Drops rows with NaNs.
    b_df_weather.dropna(axis=0, how='any', inplace=True)
            
    print('shape={}'.format(b_df_weather.shape))
        
    return b_df_weather

In [8]:
# Loads meter_reading data
def load_meter_data(data_folder_path):
    return pd.read_csv(data_folder_path + 'train.csv', parse_dates=['timestamp'])

In [9]:
# Selects only meter data from a specific building and meter id. 
def filter_meter_data(df, building_id, meter_id):
    
    to_keep = (df['building_id']==building_id) & (df['meter']==meter_id)
    b_df_meters = df[to_keep].copy()

    b_df_meters.drop('building_id', axis=1, inplace=True)
    b_df_meters.drop('meter', axis=1, inplace=True)

    b_df_meters.set_index('timestamp', inplace=True)
    b_df_meters.sort_index(inplace=True)
    
    return b_df_meters

In [10]:
"""
Drops rows that are not in both dtaframe indexes.
Converts Y from pd.df to pd.Series
"""

def prepare_meter_train_set(site_weather_df, building_meter_df):
    
    common_index = site_weather_df.index.intersection(other=building_meter_df.index)
    
    # Reset indexes
    
    X = site_weather_df.loc[common_index].copy()
    Y = building_meter_df.loc[common_index].copy()

    return (X, Y['meter_reading'])

<b>Model functions</b>

In [11]:
class MeanByMultiCatEstimator(BaseEstimator):
    """ A template estimator to be used as a reference implementation.
    For more information regarding how to build your own estimator, read more
    in the :ref:`User Guide <user_guide>`.
    Parameters
    ----------
    demo_param : str, default='demo_param'
        A parameter used for demonstation of how to pass and store paramters.
    """
    def __init__(self, cat_column_indexes=[0], verbose=False):
        self.verbose = verbose
        self.cat_column_indexes = cat_column_indexes

    def fit(self, X, y):
        """A reference implementation of a fitting function.
        Parameters
        ----------
        X : {array-like, sparse matrix}, shape (n_samples, n_features)
            The training input samples.
        y : array-like, shape (n_samples,) or (n_samples, n_outputs)
            The target values (class labels in classification, real numbers in
            regression).
        Returns
        -------
        self : object
            Returns self.
        """
        
        X, y = check_X_y(X, y, accept_sparse=True)
        """Input validation for standard estimators.
        Checks X and y for consistent length, enforces X to be 2D and y 1D. By
        default, X is checked to be non-empty and containing only finite values.
        Standard input checks are also applied to y, such as checking that y
        does not have np.nan or np.inf targets. For multi-label y, set
        multi_output=True to allow 2D and sparse y. If the dtype of X is
        object, attempt converting to float, raising on failure.
        """
        
        
        cat_columns = []
        
        for col_idx in self.cat_column_indexes:
            if(col_idx >= X.shape[1]):
                raise ValueError("category column indexes should be < X.shape[1]")
            cat_columns.append(X[:, col_idx])
            
        cat_tuples = set(zip(*cat_columns))
        
        categories = {}
        self.means = {}
        
        self.mean = y.mean()
        
        for x_bin in cat_tuples:
            categories[x_bin] = []
            
        if self.verbose:    
            print('categories : {}'.format(categories.keys()))
            
        for k in range(X.shape[0]):
            sample_bin = tuple(X[k, self.cat_column_indexes])
            categories[sample_bin].append(y[k])
        
        for k, v in categories.items():
            self.means[k] = np.array(v).mean()
        
        self.is_fitted_ = True
        # `fit` should always return `self`
        
        if self.verbose:
            for k, v in self.means.items():
                print('({}, {})'.format(k, v))
        
        return self

    
    
    def predict(self, X):
        """ A reference implementation of a predicting function.
        Parameters
        ----------
        X : {array-like, sparse matrix}, shape (n_samples, n_features)
            The training input samples.
        Returns
        -------
        y : ndarray, shape (n_samples,)
            Returns an array of ones.
        """
        
        
        X = check_array(X, accept_sparse=True)
        """Input validation on an array, list, sparse matrix or similar.
        By default, the input is checked to be a non-empty 2D array containing
        only finite values. If the dtype of the array is object, attempt
        converting to float, raising on failure."""
        
        check_is_fitted(self, 'is_fitted_')
        
        predictions = []
        
        
        cat_columns=[]
        for col in self.cat_column_indexes:
            cat_columns.append(X[:, col])
            
        cat_tuples = list(zip(*cat_columns))
        
        
        
        for sample_cat in cat_tuples:
            cat_mean = self.means.get(sample_cat)
            if(cat_mean == None):
                predictions.append(self.mean)
            else:
                predictions.append(cat_mean)
            
        
        
        return np.array(predictions)

<b>Main</b>

In [12]:
bdata.head()

,site_id
building_id,
0,0
1,0
2,0
3,0
4,0


In [13]:
site_ids = bdata['site_id'].unique().tolist()

In [14]:
train_df = pd.read_csv('../data/raw/csvs/train.csv', parse_dates=['timestamp'])

In [15]:
train_df_grouped = train_df.groupby(['building_id', 'meter']).count()
train_df_grouped.drop('timestamp', axis=1, inplace=True)
train_df_grouped.rename({'meter_reading' : 'n_meter_readings'}, axis=1, inplace=True)
train_df_grouped.sort_values(by='n_meter_readings', axis=0, ascending=False, inplace=True)

In [16]:
train_df_grouped = train_df_grouped.join(bdata, on='building_id', how='left')

In [17]:
train_df_grouped.head()

,,n_meter_readings,site_id
building_id,meter,,
0,0,8784,0
685,0,8784,5
672,0,8784,5
673,0,8784,5
674,0,8784,5


In [18]:
# main
"""

for site in site_id:

    load_and_prepare_site_data()

    for building on this site:
        
        for meter in building_meter:
        
            load_meter_data()
            prepare_train_set()
            
            cross-validate()
            fit()
            
            save()



"""




'\n\nfor site in site_id:\n\n    load_and_prepare_site_data()\n\n    for building on this site:\n        \n        for meter in building_meter:\n        \n            load_meter_data()\n            prepare_train_set()\n            \n            cross-validate()\n            fit()\n            \n            save()\n\n\n\n'

In [19]:
train_df_grouped.info()

<class 'pandas.core.frame.DataFrame'>
MultiIndex: 2380 entries, (0, 0) to (403, 0)
Data columns (total 2 columns):
n_meter_readings    2380 non-null int64
site_id             2380 non-null int64
dtypes: int64(2)
memory usage: 55.6 KB


In [20]:
train_df_grouped.iloc[:2000].tail()

,,n_meter_readings,site_id
building_id,meter,,
1003,3,8306,10
815,0,8255,8
810,0,8255,8
853,0,8208,8
1273,1,8194,14


In [21]:
# sub-sample train_df

np.random.seed(102)

# sample among first 2000 meters (with the most observations by meters)
subsample_indexes = np.random.choice(2000, 6, replace=False)

subsample_building_meters = train_df_grouped.iloc[subsample_indexes]

In [22]:
subsample_building_meters

,,n_meter_readings,site_id
building_id,meter,,
723,0,8784,5
969,1,8778,9
1242,3,8784,14
682,0,8784,5
346,0,8782,3
189,0,8781,2


In [23]:
subsample_building_meters.index.get_level_values('building_id').unique().tolist()

[723, 969, 1242, 682, 346, 189]

In [24]:
subsample_site_ids = subsample_building_meters['site_id'].unique()
subsample_site_ids

array([ 5,  9, 14,  3,  2])

In [25]:
# TODO build a specific container object for dumping

In [30]:
"""
Does not perform grid-search.
Saves models.
"""

data_folder = '../data/raw/csvs/'

tot_start_time = time()

meter_data_df = load_meter_data(data_folder)

results = []


# Create directory to save models
base_directory_path = '../models/test/'
timed_base_folder_name = 'trained_models_' + strftime('%Y%m%d_%H%M%S', localtime())
folder_path = os.path.join(base_directory_path, timed_base_folder_name)
os.mkdir(folder_path)

for site_id in subsample_site_ids:
    
    print('.site {}'.format(site_id))
    
    site_weather_data = load_and_prepare_site_data(site_id, data_folder)
    
    # GapKFold
    # gap ~ two weeks, train = 1 month (12 folds)
    gap = 24*7*2
    gap_kf = GapKFold(n_splits=12, gap_before=gap, gap_after=gap)
    
    site_building_meters = subsample_building_meters[subsample_building_meters['site_id']==site_id]
    site_buildings = site_building_meters.index.get_level_values('building_id').unique().tolist()
    #site_buildings.sort_values(by='building_id', axis='index', inplace=True)
    
    print(site_buildings)
    
    # TODO : for (building, meter) in sub_test_df_grouped.index:
    # TODO : save 2 best models
    for building in site_buildings:
        
        building_meters = site_building_meters.loc[building].index.tolist()
        
        # Create building directory
        building_folder_name = 'building_' + str(building)
        print('folder_path: {}'.format(folder_path))
        building_folder_path = os.path.join(folder_path, building_folder_name)
        print('folder_path: {}'.format(folder_path))
        print('building_folder_path: {}'.format(building_folder_path))
        os.mkdir(building_folder_path)
        
        for building_meter in building_meters:
            
            # Create meter directory
            meter_folder_name = 'meter_' + str(building_meter)
            meter_folder_path = os.path.join(building_folder_path, meter_folder_name)
            os.mkdir(meter_folder_path)
            
            print('\t.(building, meter)=({}, {})'.format(building, building_meter))
            
            load_meter_start_time = time()
            meter_df = filter_meter_data(meter_data_df, building, building_meter)
            load_meter_end_time = time()
            print('\tload_meter time : %s seconds' % (load_meter_end_time-load_meter_start_time))
            
            print('\t\tmeter_df.shape={}'.format(meter_df.shape))
            
            x_train, y_train = prepare_meter_train_set(site_weather_data, meter_df)
            
            models_and_scores = []
            
            # Dummy estimator cross-validation score
            
            dummy_regressor = DummyRegressor(strategy="mean")
            dummy_score = cross_val_score(
                estimator=dummy_regressor,
                X=x_train,
                y=y_train,
                scoring='neg_mean_squared_log_error',
                cv=gap_kf
            ).mean()
            
            dummy_regressor.fit(x_train, y_train)
            
            models_and_scores.append((dummy_regressor, dummy_score, 'dummy'))
            
            print('\t\tdummy_score={}'.format(dummy_score))
            
            # Time-only model cross-validation score
            
            day_hour_col_idx = x_train.columns.to_list().index('day_hour')
            day_of_week_col_idx = x_train.columns.to_list().index('day_of_week')
            time_col_indexes = [day_hour_col_idx, day_of_week_col_idx]
            
            time_avg_model = MeanByMultiCatEstimator(time_col_indexes)
            time_avg_score = cross_val_score(
                estimator=time_avg_model,
                X=x_train,
                y=y_train,
                scoring='neg_mean_squared_log_error',
                cv=gap_kf
            ).mean()
            
            time_avg_model.fit(x_train, y_train)
            
            models_and_scores.append((time_avg_model, time_avg_score, 'time_avg'))
            
            print('\t\ttime_avg_score={}'.format(time_avg_score))
            
            # Time + weather random forest model
            
            no_gcv_start_time = time()
            
            rfr_model = RandomForestRegressor(n_estimators=70, max_depth=10)
            
            rfr_no_gcv_score = cross_val_score(
                estimator=rfr_model,
                X=x_train,
                y=y_train,
                scoring='neg_mean_squared_log_error',
                cv=gap_kf
            ).mean()
            
            no_gcv_end_time = time()
            print('\t\tno_gcv time : %s seconds' % (no_gcv_end_time-no_gcv_start_time))
            
            
            
            # fit model
            rfr_model.fit(X=x_train, y=y_train)
            
            models_and_scores.append((rfr_model, rfr_no_gcv_score, 'rfr'))
            
            # Find best model
            models, scores, names = list(zip(*models_and_scores))
            scores_argmax = np.argmax(np.array(scores))
            best_model = models[scores_argmax]
            best_model_name = names[scores_argmax]
            
            # save best model
            model_path = os.path.join(meter_folder_path, 'best_model.joblib')
            
            joblib.dump(best_model, model_path)
            
            
            # training info savings in df 
            rfr_improvement_no_gcv = - (rfr_no_gcv_score-time_avg_score) / time_avg_score * 100 ;
            
            result_row = [
                building, 
                building_meter,
                rfr_improvement_no_gcv,
                best_model_name
            ]
            
            results.append(result_row)
            
            
    print('--')
    
tot_end_time = time()
print('total time : %s seconds' % (tot_end_time-tot_start_time))

col_names = ['building', 'meter_id', 'rfr_improvement', 'saved_model']
results_df = pd.DataFrame(results, columns=col_names)

results_df.to_csv(os.path.join(folder_path, 'training_info.csv'), index=False)

.site 5
shape=(8707, 6)
[723, 682]
folder_path: ../models/test/trained_models_20200328_173456
folder_path: ../models/test/trained_models_20200328_173456
building_folder_path: ../models/test/trained_models_20200328_173456/building_723
	.(building, meter)=(723, 0)
	load_meter time : 0.08563899993896484 seconds
		meter_df.shape=(8784, 1)
		dummy_score=-3.262917301171566
		time_avg_score=-3.1095751678833508
		no_gcv time : 9.914153099060059 seconds
folder_path: ../models/test/trained_models_20200328_173456
folder_path: ../models/test/trained_models_20200328_173456
building_folder_path: ../models/test/trained_models_20200328_173456/building_682
	.(building, meter)=(682, 0)
	load_meter time : 0.05803418159484863 seconds
		meter_df.shape=(8784, 1)
		dummy_score=-0.22292376616670864
		time_avg_score=-0.11042779775559657
		no_gcv time : 9.936627864837646 seconds
--
.site 9
shape=(8760, 6)
[969]
folder_path: ../models/test/trained_models_20200328_173456
folder_path: ../models/test/trained_models

In [27]:
results_df

,building,meter_id,rfr_improvement,saved_model
0,723,0,-8.481030,time_avg
1,682,0,32.027189,rfr
2,969,1,46.805290,rfr
3,1242,3,63.414382,rfr
4,346,0,-20.479436,time_avg
5,189,0,-71.728734,time_avg


In [28]:
model_id = {0 : 'dummy', 1 : 'time_avg', 2 : 'rfr'}
model_id[1]

'time_avg'

In [29]:
models_and_scores

[(DummyRegressor(constant=None, quantile=None, strategy='mean'),
  -0.1274181424041007,
  'dummy'),
 (MeanByMultiCatEstimator(cat_column_indexes=[2, 3], verbose=False),
  -0.10677607657629619,
  'time_avg'),
 (RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=10,
                        max_features='auto', max_leaf_nodes=None,
                        min_impurity_decrease=0.0, min_impurity_split=None,
                        min_samples_leaf=1, min_samples_split=2,
                        min_weight_fraction_leaf=0.0, n_estimators=70,
                        n_jobs=None, oob_score=False, random_state=None,
                        verbose=0, warm_start=False),
  -0.18336520428597466,
  'rfr')]